In [1]:
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.7 MB 8.2 MB/s eta 0:00:01


In [3]:
!pip install tables

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 4.3 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 162 kB 15.9 MB/s eta 0:00:01


In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from tqdm import tqdm

In [2]:
# test_data = pd.read_csv('test_data_B.csv',sep = '|')
# test_data.to_hdf('test_data_B.h5', key='data')

In [3]:
# train_data = pd.read_csv('train_data.csv',sep = '|')
# test_data = pd.read_csv('test_data_A.csv',sep = '|')
# train_data.to_hdf('train_data.h5', key='data')
# test_data.to_hdf('test_data.h5', key='data')
train_data = pd.read_hdf('train_data.h5', key='data')
test_data_A = pd.read_hdf('test_data.h5', key='data')
test_data_B = pd.read_hdf('test_data_B.h5', key='data')

In [4]:
test_data_B['pt_d']-=1

In [5]:
test_data = pd.concat([test_data_A,test_data_B])

In [6]:
test_data.shape

(2000000, 36)

In [7]:
train_data['min_onlinerate'] = train_data['communication_onlinerate'].apply(lambda x:int(x.split('^')[0]))
train_data['max_onlinerate'] = train_data['communication_onlinerate'].apply(lambda x:int(x.split('^')[-1]))
test_data['min_onlinerate'] = test_data['communication_onlinerate'].apply(lambda x:int(x.split('^')[0]))
test_data['max_onlinerate'] = test_data['communication_onlinerate'].apply(lambda x:int(x.split('^')[-1]))
#0.764802

In [8]:
temp = train_data.groupby(['uid','pt_d']).agg({'label':'sum'}).reset_index()
temp.columns = ['uid','pt_d','click_lastday_count']
temp['pt_d']+=1
test_data = test_data.merge(temp,on = ['uid','pt_d'],how = 'left')
train_data = train_data.merge(temp,on = ['uid','pt_d'],how = 'left')
#+0.1%

In [9]:
temp = train_data.groupby(['uid','pt_d']).agg({'adv_id':'count'}).reset_index()
temp.columns = ['uid','pt_d','expo_lastday_counts']
temp['pt_d']+=1
test_data = test_data.merge(temp,on = ['uid','pt_d'],how = 'left')
train_data = train_data.merge(temp,on = ['uid','pt_d'],how = 'left')
#001 0.776932 - 0.758022   # +3.7%

In [10]:
temp = test_data.groupby(['uid','pt_d']).agg({'adv_id':'count'}).reset_index()
temp.columns = ['uid','pt_d','expo_day_counts']
test_data = test_data.merge(temp,on = ['uid','pt_d'],how = 'left')

temp = train_data.groupby(['uid','pt_d']).agg({'adv_id':'count'}).reset_index()
temp.columns = ['uid','pt_d','expo_day_counts']
train_data = train_data.merge(temp,on = ['uid','pt_d'],how = 'left')
#002 79 0.751055 +6%  +3%

In [12]:
test_data['expo_day_counts']*=3 #0.781291

In [13]:
param = {'num_leaves': 52,
         'objective':'binary',
         'learning_rate': 0.1,        
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 133,
         "verbosity": -1,
         "num_threads" : -1,
#          'device': 'gpu',
#          'gpu_platform_id': 0,
#          'gpu_device_id': 0
         }

In [14]:
train_data.columns

Index(['label', 'uid', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'age', 'city', 'city_rank',
       'device_name', 'device_size', 'career', 'gender', 'net_type',
       'residence', 'his_app_size', 'his_on_shelf_time', 'app_score',
       'emui_dev', 'list_time', 'device_price', 'up_life_duration',
       'up_membership_grade', 'membership_life_duration', 'consume_purchase',
       'communication_onlinerate', 'communication_avgonline_30d', 'indu_name',
       'pt_d', 'min_onlinerate', 'max_onlinerate', 'click_lastday_count',
       'expo_lastday_counts', 'expo_day_counts'],
      dtype='object')

In [15]:
all_col=['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'age', 'city', 'city_rank',
       'device_name', 'device_size', 'career', 'gender', 'net_type',
       'residence', 'his_app_size', 'his_on_shelf_time', 'app_score',
       'emui_dev', 'list_time', 'device_price', 'up_life_duration',
       'up_membership_grade', 'membership_life_duration', 'consume_purchase',
        'communication_avgonline_30d', 'indu_name',
         'expo_day_counts','expo_lastday_counts','click_lastday_count','min_onlinerate','max_onlinerate',
        ]# ,

In [16]:
cat_col=['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'city', 'city_rank',
       'device_name', 'device_size', 'career', 'gender', 'net_type',
       'residence', 'his_app_size', 'his_on_shelf_time', 'app_score',
       'emui_dev', 'list_time', 'device_price', 'up_life_duration',
       'up_membership_grade', 'membership_life_duration', 'consume_purchase',
        'communication_avgonline_30d', 'indu_name']# 

In [17]:
target = train_data['label']
train_data = train_data[all_col]
test_data = test_data[all_col]

In [18]:
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,RepeatedKFold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)
stack_test = np.zeros((len(test_data.iloc[1000000:]),1))
stack_train = np.zeros((len(train_data),1))

In [19]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_data, target)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_data.iloc[trn_idx],
                           label=target[trn_idx],
                           categorical_feature=cat_col
                           )
    val_data = lgb.Dataset(train_data.iloc[val_idx],
                           label=target[val_idx],
                           categorical_feature=cat_col
                           )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=50,
                    early_stopping_rounds=100)
    stack_train[val_idx, :] =clf.predict(train_data.iloc[val_idx], num_iteration=clf.best_iteration).reshape(-1,1)
    stack_test += clf.predict(test_data.iloc[1000000:], num_iteration=clf.best_iteration).reshape(-1,1)
    

fold n°0


/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
[50]	training's auc: 0.802754	valid_1's auc: 0.799858
[100]	training's auc: 0.807458	valid_1's auc: 0.8011
[150]	training's auc: 0.810451	valid_1's auc: 0.801404
[200]	training's auc: 0.812894	valid_1's auc: 0.801531
[250]	training's auc: 0.815069	valid_1's auc: 0.801624
[300]	training's auc: 0.816914	valid_1's auc: 0.801628
[350]	training's auc: 0.818612	valid_1's auc: 0.801559
Early stopping, best iteration is:
[287]	training's auc: 0.816429	valid_1's auc: 0.801643
fold n°1
Training until validation scores don't improve for 100 rounds
[50]	training's auc: 0.80283	valid_1's auc: 0.799345
[100]	training's auc: 0.807449	valid_1's auc: 0.800652
[150]	training's auc: 0.810373	valid_1's auc: 0.800956
[200]	training's auc: 0.812885	valid_1's auc: 0.801117
[250]	training's auc: 0.815096	valid_1's auc: 0.801198
[300]	training's auc: 0.817051	valid_1's auc: 0.801183
[350]	training's auc: 0.818713	valid_1's auc: 0.801171
Early stoppi

In [21]:
oof_train = pd.DataFrame(stack_train)
oof_test = pd.DataFrame(stack_test/5)
pd.concat([oof_train,oof_test]).to_hdf('lgboof.h5', key='data')

In [22]:
feature_importance_df = pd.DataFrame()
feature_importance_df["importance"] = clf.feature_importance(importance_type='gain')

In [23]:
feature_importance_df["feature"] = all_col

In [22]:
feature_importance_df.sort_values('importance',ascending=False)

NameError: name 'feature_importance_df' is not defined